In [2]:
import pandas as pd
import sqlite3
import os, sys
from sklearn.preprocessing import MinMaxScaler
import torch
import numpy as np
import time

idx = os.getcwd().index("trade")
PROJECT_HOME = os.getcwd()[:idx] + "trade/"
sys.path.append(PROJECT_HOME)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from common.global_variables import *
from upbit.upbit_api import Upbit
from db.sqlite_handler import *

from db.sqlite_handler import *
from upbit.upbit_order_book_based_data import UpbitOrderBookBasedData, get_data_loader
from skorch import NeuralNetClassifier
from predict.model_rnn import LSTM
import predict

In [3]:
upbit_order_book_data = UpbitOrderBookBasedData("ADA")

In [4]:
x_normalized_original, y_up_original, one_rate, total_size = upbit_order_book_data.get_dataset(split=False)

In [17]:
def make_lstm_model(coin_name, x_normalized_original, y_up_original, total_size, one_rate):
    lstm_model = LSTM(input_size=INPUT_SIZE).to(DEVICE)
    
    net = NeuralNetClassifier(
        lstm_model,
        max_epochs=10,
        lr=0.1,
        # Shuffle training data on each epoch
        iterator_train__shuffle=True,
    )

    X = x_normalized_original
    y = y_up_original.type(torch.LongTensor) 
    print(y)
    
    print(type(X), type(y))
    print(X.shape, y.shape)
#    print(X.size(), y.size())        
#    print(X.dim(), y.dim())    
    
    net.fit(X=X, y=y)

In [18]:
best_model = make_lstm_model("ADA", x_normalized_original, y_up_original, total_size, one_rate)

tensor([0, 0, 0,  ..., 1, 1, 1])
<class 'torch.Tensor'> <class 'torch.Tensor'>
torch.Size([1066, 18, 125]) torch.Size([1066])


RuntimeError: Assertion `cur_target >= 0 && cur_target < n_classes' failed.  at ../aten/src/THNN/generic/ClassNLLCriterion.c:92